In [1]:
import pandas as pd
from resource_consolidation import Resources, Trip, DailyMetrics
from json import dumps

In [2]:
url = 'https://data.wprdc.org/dataset/healthyride-trip-data'
resource_extension = '/dataset/healthyride-trip-data/resource/'
resource_calling_string = 'https://data.wprdc.org/api/3/action/datastore_search?resource_id='


In [3]:
#create empty container for data 
data_container = Resources(url, resource_extension, resource_calling_string)
#create json string with data from API
json_list = data_container.create_json()

working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=7fe58023-39be-4fa5-a7b7-27d7b0e077a7
of length: 15100
correct length
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=a077921d-ae97-48fd-a85d-b31395245ae1
of length: 29811
correct length
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=fcf1fe7c-b59f-4873-8b66-3715191af75a
of length: 8813
correct length
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=030d12a3-fda6-4f2c-a508-ff23a4ecf195
of length: 26973
correct length
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=6ab85d5c-b7a1-41f2-a7a5-1eec8cea4d8c
of length: 19855
correct length
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=c85436af-ba38-4d91-967f-3e11e4d62a43
of length: 500000
correct length
working on: https://data.wprdc.org/api/3/action/datastore_search?resource_id=fe985614-775c-42e3-9647-59099806329d
of length:

In [4]:
print(json_list[0])


{'Trip id': '14321596', 'Bikeid': '70374', 'To station name': 'Boulevard of the Allies & Parkview Ave', 'Usertype': 'Subscriber', 'Stoptime': '10/1/2015 0:32', 'From station name': 'Fifth Ave & S Dithridge St', 'Starttime': '10/1/2015 0:22', 'To station id': '1038', 'Tripduration': '572', '_id': 1, 'From station id': '1035'}


In [5]:
#convert list of dicts to list of objects called "Trips"
Trip_list = []
for dict in json_list:
    li = list(dict.values())
    Trip_list.append(Trip(*li))

In [6]:
#test er out
print(Trip_list[420])
print(Trip_list[420].toname)
print(Trip_list[420].starttime)

Fifth Ave & S Bouquet St
10/2/2015 11:56


In [60]:
pncstation_trip_list_to = [i for i in Trip_list if i.toid == '1013']
pncstation_trip_list_from = [i for i in Trip_list if i.fromid == '1013']


print(len(pncstation_trip_list_to))
print(len(pncstation_trip_list_from))


17461
16643


In [66]:
class Day:
    #date is the day, arrivals is list of arrivals, departures is list of departures
    def __init__ (self, date, arrivals = [], departures = []):
        self.date = date
        self.arrivals = arrivals
        self.departures = departures

In [68]:
def departure_params_todict(list_of_trip_objects):

    previous_day = list_of_trip_objects[0].starttime.split(" ")[0]
    param_li = []
    daily_dict = {}
    for trip in list_of_trip_objects:
        this_day = trip.starttime.split(" ")[0]
        if previous_day != this_day:
            daily_dict[previous_day]= param_li
            param_li = []
        else:
            param_li.append(trip.starttime.split(" ")[1])
        previous_day = this_day
    return daily_dict

def arrival_params_todict(list_of_trip_objects):
    
    previous_day = list_of_trip_objects[0].starttime.split(" ")[0]
    param_li = []
    daily_dict = {}
    for trip in list_of_trip_objects:
        this_day = trip.stoptime.split(" ")[0]
        if previous_day != this_day:
            daily_dict[previous_day]= param_li
            param_li = []
        else:
            param_li.append(trip.stoptime.split(" ")[1])
        previous_day = this_day
    return daily_dict


In [69]:
to_dict = arrival_params_todict(pncstation_trip_list_to)
from_dict = departure_params_todict(pncstation_trip_list_from)

In [80]:
to_dates = list(to_dict.keys())
from_dates = list(from_dict.keys())

unique_to_arrivals = [i for i in to_dates if i not in from_dates]
unique_to_departures = [i for i in from_dates if i not in to_dates]

shared_dates = [i for i in to_dates if i not in unique_to_arrivals]

In [84]:
Day_object_list = []

for i in shared_dates:
    Day_object_list.append(Day(i, to_dict[i], from_dict[i]))



In [98]:
search_index = shared_dates.index('8/3/2019')

print(Day_object_list[search_index].date)
print(Day_object_list[search_index].arrivals)
print(Day_object_list[search_index].departures)

8/3/2019
['23:14', '23:14', '23:18', '23:13']
['7:57', '9:11', '9:12', '13:03', '16:43', '17:06', '17:06', '17:07', '17:13', '17:21', '17:42', '17:44', '18:35', '19:24', '19:24', '19:24', '19:24', '20:17', '20:17', '21:53', '21:59', '22:03', '23:01', '23:01', '23:01', '23:01', '23:15', '23:16', '23:17', '23:22']


Unused Functions Below

In [56]:
def daily_parameters (trip_list):
    daily_dict = {}
    previous_day = trip_list[0].starttime[:8]
    i = 0
    for trip in trip_list:
        this_day = trip.starttime.split(" ")[0]
        if previous_day != trip.starttime[:8]:
            daily_dict[previous_day] = i
            i = 0
        else:
            i += 1
        previous_day = this_day
    return daily_dict

In [57]:
#create json string from json list
jsonStr = dumps(json_list)
#create dataframe with data from json string
df = pd.read_json(jsonStr)
#double checking head of dataframe to see if it correctly combined
print(df.head(n=10))

TypeError: unhashable type: 'dict'